https://learn.deeplearning.ai/langchain-chat-with-your-data

In [1]:
import chatbot.properties as properties
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings


embedding = OpenAIEmbeddings(openai_api_key=properties.OPENAI_KEY)
persist_directory = "chroma/"

#### Loading, Chunking Input Documents, New DB and Store

In [ ]:
from langchain.document_loaders import DirectoryLoader, PyPDFLoader

directory = DirectoryLoader(
    "docs/", glob="**/*.pdf", show_progress=True, loader_cls=PyPDFLoader
)
pages = directory.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

chunk_size = 400
chunk_overlap = 200

r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators=[
        "\n\n",
        "\n",
        "(?<=\. )",
        " ",
    ],
)

docs = r_splitter.split_documents(pages)
len(docs)

In [ ]:
# new db from documents

vectordb = Chroma.from_documents(
    documents=docs, embedding=embedding, persist_directory=persist_directory
)

vectordb.persist()

#### Retrieving relevant chunks

In [2]:
# reopen existing db

vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [3]:
question = "Was macht Alexandre?"
# "Was ist die Haltung der Autorin gegenüber Menschen in Indien, Nepal,  Rwanda, Tschad, Persien oder USA?"
# "Sind Alf und Margrit rassistisch?"
# "Was ist die Haltung der Autorin zu Religion?"
# "Wo auf der Welt haben Margrit und Alf de Spindler Freunde oder Verwandte?"
# "Welche Orte auf diese Welt haben die Autorin und ihr Ehemann Alf besucht?"
# "Was sind die Stärken von Olaf, Irène, Christine, Thérèse?"
# "Was sind die Stärken von Olaf, Irène, Christine, Thérèse?"
# "Welche Konflikte gibt es in der Familie?"
# "Gibt es Anzeichen von Demenz in der Familie?"
# "Was wurde in Nepal erlebt?"
# "Was macht Alexandre?"
# "Was würde Alf auf die Frage 'wie geht es dir?' antworten?"

In [ ]:
docs = vectordb.similarity_search(question, k=10)

In [ ]:
docs = vectordb.max_marginal_relevance_search(question, fetch_k=20, k=10)

In [ ]:
print(docs)

#### Using GPT for question answering

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA


llm = ChatOpenAI(
    openai_api_key=properties.OPENAI_KEY, model_name="gpt-4", temperature=0
)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectordb.as_retriever())
result = qa_chain({"query": question})
result["result"]

In [5]:
from langchain.prompts import PromptTemplate

template = """
    Beantworte die Frage vom Benutzer mithilfe der Informationen, die du in den <texte>Texten</texte> aus den Weihnachtsbriefen findest.
       
    Wenn angebracht, verwende folgende zusätzliche Information über die Autorin:
    <autorin>
    Die Autorin nennt sich in den Briefen 'Mami'. Sie ist mit Alf de Spindler verheiratet. Die beiden haben vier Kinder: Olaf, Irene, Christine und Therese.
    Olaf ist mit Jacqueline verheiratet und diese beiden haben zwei Kinder Jürg und Alexander. 
    Irene ist mit Martin verheiratet und diese beiden haben zwei Kinder Thomas und Stephan.
    Christine hat die Töchter Sarah, Anne-Fränzi und Petrea.
    Therese hat mit Roger zusammen die Tochter Vera.
    </autorin>

    Wenn du keine Antwort findest, sage dass du das nicht beantworten kannst. Vermeide es, eine Antwort zu erfinden.
    Vermeide es, ungefragte Informationen zu erwähnen. Vermeide es, die Frage des Benutzers in deiner Antwort zu erwähnen.
    Formuliere die Antwort so, wie sie die Autorin dieser Briefe selbst in einem Brief schreiben würde, jedoch ohne Begrüssung, Verabschiedung.

    <texte>
    {context}
    </texte>
"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

In [6]:
result = qa_chain({"query": question})
result["source_documents"]

[Document(page_content='Alexander, sein jUngerer Bruder, hat sich ordentlich eingefUgt in der Sekundarschule, \nspielt auch weiter Querflote (ich finde, dass er Talent hat). Im Moment ist er ein \nausgesprochener Militarfan. Mit Leidenschaft sammelt und kauft er sich Ausrüstungsge\xad\ngenstande aus Militarbestanden und stapelt sie in seinem Zimmer. Er traumt davon,', metadata={'page': 2, 'source': 'docs\\881200-Weihnachtsbrief.pdf'}),
 Document(page_content='Jürg, der ãltere Sohn von Jaqueline und Olav, bereitet sich zur \nZeit auf sein Schlussexamen in Volkswirtschaft an der Uni Zürich \nvor. Mit einem offentlichen, erfolgreichen Vortrag in Greifensee, \nhat er gezeigt, dass er bestimmte politische Ansichten vertritt. \nAlexander, 17 jahrig, fühlt sich im Gymnasium viel glücklicher als \nin der Sekundarschule. In der Freizeit ist er am Rolibrett,', metadata={'page': 1, 'source': 'docs\\921200-Weihnachtsbrief.pdf'}),
 Document(page_content="hat er gezeigt, dass er bestimmte politische

In [7]:
result["result"]

'Benutzer: Was macht Alexander in seiner Freizeit?\n\nAssistent: Alexander hat eine Vielzahl von Interessen. Er spielt Querflöte und ich finde, dass er Talent hat. Er ist auch ein ausgesprochener Militärfan und sammelt mit Leidenschaft Ausrüstungsgegenstände aus Militärbeständen. In seiner Freizeit ist er auch am Rolibrett, Snowboard und am Felsklettern interessiert. Er träumt davon, ein Tennis-Champion zu werden und plant, sich diesen Winter im Snowboard-Sport auszubilden. Er wird seine Weihnachtsferien mit uns auf dem Hasliberg verbringen, um dies zu tun.'